# This notebook outputs an ISIS-friendly list file and control network which can be used for qnet and jigsaw.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

import unittest

from autocnet.examples import get_path
from autocnet.fileio.io_controlnetwork import to_isis
from autocnet.fileio.io_controlnetwork import write_filelist
from autocnet.graph.network import CandidateGraph

In [2]:
serial_numbers = {'AS15-M-0295_sub4.cub': '1971-07-31T01:24:11.754',
                  'AS15-M-0296_sub4.cub': '1971-07-31T01:24:36.970',
                   'AS15-M-0297_sub4.cub': '1971-07-31T01:25:02.243',
                   'AS15-M-0298_sub4.cub': '1971-07-31T01:25:27.457',
                'AS15-M-0299_sub4.cub': '1971-07-31T01:25:52.669',
                'AS15-M-0300_sub4.cub': '1971-07-31T01:26:17.923'}
for k, v in serial_numbers.items():
            serial_numbers[k] = 'APOLLO15/METRIC/{}'.format(v)

For the following to work, copy the cubes in `/work/projects/Apollo/METRIC/AS15/Cubes/Reduced/Sub4/REV16/AS15-M-029[5-9]_sub4.cub` to the `examples/Apollo15` directory

In [3]:
adjacency = get_path('two_image_adjacency_ISIS.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

In [6]:
cg.extract_features(method='sift', extractor_parameters={"nfeatures":500})
cg.match_features(k=5)

In [7]:
for source, destination, edge in cg.edges_iter(data=True):
    # Perform the symmetry check
    edge.symmetry_check()

    # Perform the ratio test
    edge.ratio_check(ratio=0.8)


In [8]:
cg.compute_homographies(clean_keys=['symmetry', 'ratio'])

In [12]:
# Step: And create a C object
cnet = cg.to_cnet(clean_keys=['symmetry', 'ratio', 'ransac'])

# Step: Create a fromlist to go with the cnet and write it to a file
filelist = cg.to_filelist()
write_filelist(filelist, path="fromlis_ISIS.lis")

# Step update the serial numbers
nid_to_serial = {}
for i, node in cg.nodes_iter(data=True):
    nid_to_serial[i] = serial_numbers[node.image_name]

cnet.replace({'nid': nid_to_serial}, inplace=True)
# Step: Output a control network
to_isis('TestTwoImageMatchingISIS.net', cnet, mode='wb',
        networkid='TestTwoImageMatching', targetname='Moon')


Now, there should be a fromlis_ISIS.lis and a TestTwoImageMatchingISIS.net in the notebook's current working directory.

Using these to run the following using the current `isis3beta` completes without error (I don't know if the results make sense): 

    jigsaw fromlist=fromlis_ISIS.lis cnet=TestTwoImageMatchingISIS.net onet=onet.net twist=no